In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import os
import seaborn as sns
import pyarrow.parquet as pq #reading parquet files 

In [ ]:
# dataset upload
INIT_DIR = '../input'
SIZE = 2001
train = pq.read_pandas(os.path.join(INIT_DIR, 'vsb-power-line-fault-detection/train.parquet'), columns=[str(i) for i in range(SIZE)]).to_pandas()
metadata = pd.read_csv('../input/vsb-power-line-fault-detection/metadata_train.csv')

In [ ]:
train_metadata = metadata[:SIZE]
train = train.T
train['signal_id'] = list(train_metadata['signal_id'])
train = train.merge(train_metadata, on='signal_id')

In [ ]:
phase1_normal=train.loc[0].values
phase2_normal=train.loc[1].values
phase3_normal=train.loc[2].values

phase1_faulty=train.loc[3].values
phase2_faulty=train.loc[4].values
phase3_faulty=train.loc[5].values

dt=(20e-3)/len(phase1_faulty)
fs=1/dt
t=np.linspace(0,len(phase1_normal)*(1/fs),len(phase1_normal))

In [ ]:
plt.figure(figsize=(12, 6))
plt.title("normal lines")
plt.xlabel("time [ms]")
plt.plot(t,phase1_normal,'r-', alpha=0.7,label='phase 1')
plt.plot(t,phase2_normal,'k-', alpha=0.7,label='phase 2')
plt.plot(t,phase3_normal,'b-', alpha=0.7,label='phase 3')
plt.ylim(-80,60)
plt.legend()

In [ ]:
plt.figure(figsize=(12, 6))
plt.title("faulty lines")
plt.xlabel("time [ms]")
plt.plot(t,phase1_faulty,'r-', alpha=0.7,label="phase 1")
plt.plot(t,phase2_faulty,'k-', alpha=0.7,label="phase 2")
plt.plot(t,phase3_faulty,'b-', alpha=0.7,label="phase 3")
plt.ylim(-80,60)
plt.legend()

In [ ]:
from scipy.fft import fft, fftfreq,ifft,fftshift
dt=(20e-3)/len(phase1_faulty)
fs=1/dt

yf_n = fft(np.array(phase1_normal))
xf_n = fftfreq(yf_n.size, dt)

yf_f = fft(np.array(phase1_faulty))
xf_f = fftfreq(yf_f.size, dt)

fig, axes = plt.subplots(2,1, dpi=130, figsize=(4,4), sharex=True)
axes[0].semilogy(fftshift(xf_n), np.absolute(fftshift(yf_n)),'k',lw=2,label='phase 1 normal')
axes[0].legend()
axes[0].set_xlim(0,fs/2)
axes[0].set_ylim(1e1,1e7)
axes[0].set_xlabel("frequency [Hz]")
axes[0].set_ylabel("|FFT|")
axes[0].grid()

axes[1].semilogy(fftshift(xf_f), np.absolute(fftshift(yf_f)),'r',lw=2,label='phase 1 faulty')
axes[1].legend()
axes[1].set_xlim(0,fs/2)
axes[1].set_ylim(1e1,1e7)
axes[1].set_xlabel("frequency [Hz]")
axes[1].set_ylabel("|FFT|")
axes[1].grid()

In [ ]:
from scipy.fft import fft, fftfreq,ifft,fftshift
dt=(20e-3)/len(phase2_faulty)
fs=1/dt

yf_n = fft(np.array(phase2_normal))
xf_n = fftfreq(yf_n.size, dt)

yf_f = fft(np.array(phase2_faulty))
xf_f = fftfreq(yf_f.size, dt)

fig, axes = plt.subplots(2,1, dpi=130, figsize=(4,4), sharex=True)
axes[0].semilogy(fftshift(xf_n), np.absolute(fftshift(yf_n)),'k',lw=2,label='phase 2 normal')
axes[0].legend()
axes[0].set_xlim(0,fs/2)
axes[0].set_ylim(1e1,1e7)
axes[0].set_xlabel("frequency [Hz]")
axes[0].set_ylabel("|FFT|")
axes[0].grid()

axes[1].semilogy(fftshift(xf_f), np.absolute(fftshift(yf_f)),'r',lw=2,label='phase 2 faulty')
axes[1].legend()
axes[1].set_xlim(0,fs/2)
axes[1].set_ylim(1e1,1e7)
axes[1].set_xlabel("frequency [Hz]")
axes[1].set_ylabel("|FFT|")
axes[1].grid()

In [ ]:
from scipy.fft import fft, fftfreq,ifft,fftshift
dt=(20e-3)/len(phase3_faulty)
fs=1/dt

yf_n = fft(np.array(phase3_normal))
xf_n = fftfreq(yf_n.size, dt)

yf_f = fft(np.array(phase3_faulty))
xf_f = fftfreq(yf_f.size, dt)

fig, axes = plt.subplots(2,1, dpi=130, figsize=(4,4), sharex=True)
axes[0].semilogy(fftshift(xf_n), np.absolute(fftshift(yf_n)),'k',lw=2,label='phase 3 normal')
axes[0].legend()
axes[0].set_xlim(0,fs/2)
axes[0].set_ylim(1e1,1e7)
axes[0].set_xlabel("frequency [Hz]")
axes[0].set_ylabel("|FFT|")
axes[0].grid()

axes[1].semilogy(fftshift(xf_f), np.absolute(fftshift(yf_f)),'r',lw=2,label='phase 3 faulty')
axes[1].legend()
axes[1].set_xlim(0,fs/2)
axes[1].set_ylim(1e1,1e7)
axes[1].set_xlabel("frequency [Hz]")
axes[1].set_ylabel("|FFT|")
axes[1].grid()

In [ ]:
from scipy import signal
f1_n, t1_n, Sxx1_n = signal.spectrogram(phase1_normal, fs)
f1_f, t1_f, Sxx1_f = signal.spectrogram(phase1_faulty, fs)

fig=plt.figure(figsize=(14,4))
ax1=fig.add_subplot(121)
im1=ax1.pcolormesh(t1_n, f1_n, 20*np.log10(Sxx1_n), shading='gouraud')
ax1.set_title("phase 1 normal")
ax1.set_ylabel('Frequency [Hz]')
ax1.set_xlabel('Time [sec]')
fig.colorbar(im1).set_label('Intensity [dB]')
im1.set_clim(-400, -100)

ax2=fig.add_subplot(122)
im2=ax2.pcolormesh(t1_f, f1_f, 20*np.log10(Sxx1_f), shading='gouraud')
ax2.set_title("phase 1 faulty")
ax2.set_ylabel('Frequency [Hz]')
ax2.set_xlabel('Time [sec]')
fig.colorbar(im2).set_label('Intensity [dB]')
im2.set_clim(-400, -100)



In [ ]:
f2_n, t2_n, Sxx2_n = signal.spectrogram(phase2_normal, fs)
f2_f, t2_f, Sxx2_f = signal.spectrogram(phase2_faulty, fs)

fig=plt.figure(figsize=(14,4))
ax1=fig.add_subplot(121)
im1=ax1.pcolormesh(t2_n, f2_n, 20*np.log10(Sxx2_n), shading='gouraud')
ax1.set_title("phase 2 normal")
ax1.set_ylabel('Frequency [Hz]')
ax1.set_xlabel('Time [sec]')
fig.colorbar(im1).set_label('Intensity [dB]')
im1.set_clim(-400, -100)

ax2=fig.add_subplot(122)
im2=ax2.pcolormesh(t2_f, f2_f, 20*np.log10(Sxx2_f), shading='gouraud')
ax2.set_title("phase 2 faulty")
ax2.set_ylabel('Frequency [Hz]')
ax2.set_xlabel('Time [sec]')
fig.colorbar(im2).set_label('Intensity [dB]')
im2.set_clim(-400, -100)


In [ ]:
f3_n, t3_n, Sxx3_n = signal.spectrogram(phase3_normal, fs)
f3_f, t3_f, Sxx3_f = signal.spectrogram(phase3_faulty, fs)

fig=plt.figure(figsize=(14,4))
ax1=fig.add_subplot(121)
im1=ax1.pcolormesh(t3_n, f3_n, 20*np.log10(Sxx3_n), shading='gouraud')
ax1.set_title("phase 3 normal")
ax1.set_ylabel('Frequency [Hz]')
ax1.set_xlabel('Time [sec]')
fig.colorbar(im1).set_label('Intensity [dB]')
im1.set_clim(-400, -100)

ax2=fig.add_subplot(122)
im2=ax2.pcolormesh(t3_f, f3_f, 20*np.log10(Sxx3_f), shading='gouraud')
ax2.set_title("phase 3 faulty")
ax2.set_ylabel('Frequency [Hz]')
ax2.set_xlabel('Time [sec]')
fig.colorbar(im2).set_label('Intensity [dB]')
im2.set_clim(-400, -100)


In [ ]:
import numpy as np
from numpy.fft import rfft, irfft, rfftfreq
from scipy import fftpack

def decompose_into_n_signals(srs, n):
    fourier = rfft(srs)
    frequencies = rfftfreq(srs.size, d=2e-2/srs.size)
    out = []
    for vals in np.array_split(frequencies, n):
        ft_threshed = fourier.copy()
        ft_threshed[(vals.min() > frequencies)] = 0
        ft_threshed[(vals.max() < frequencies)] = 0        
        out.append(irfft(ft_threshed))
    return out

def plot_n_signals(sigs):
    fig, axarr = plt.subplots(len(sigs), figsize=(12, 12))
    for i, sig in enumerate(sigs):
        plt.sca(axarr[i])
        plt.plot(sig)
    plt.gcf().suptitle(f"Decomposition into {len(sigs)} frequency bands", fontsize=14)

In [ ]:
from scipy.fft import fft, fftfreq,ifft,fftshift

def brickwall(xf,yf,cutoff):
    for i in range(len(xf)):  
        if xf[i]>=cutoff or xf[i]<=-cutoff:
            yf[i]=0
    return yf  

In [ ]:
filt_brick_normal = ifft(brickwall(xf_n,yf_n,1.25e7))    

plt.plot(t, phase3_normal, 'b-',linewidth=3, label='data')
plt.plot(t, filt_brick_normal, 'g--', linewidth=2, label='filtered')
plt.xlabel('time [ms]')
plt.ylim(-80,60)
plt.legend()
                      

In [ ]:
filt_brick_faulty = ifft(brickwall(xf_f,yf_f,1.25e7))    

plt.plot(t, phase3_faulty, 'b-',linewidth=3, label='data faulty')
plt.plot(t, filt_brick_faulty, 'g--', linewidth=2, label='filtered')
plt.xlabel('time [ms]')
plt.ylim(-80,60)
plt.legend()

In [ ]:
f3_n, t3_n, Sxx3_n = signal.spectrogram(filt_brick_normal, fs)
f3_f, t3_f, Sxx3_f = signal.spectrogram(filt_brick_faulty, fs)

fig=plt.figure(figsize=(14,4))
ax1=fig.add_subplot(121)
im1=ax1.pcolormesh(t3_n, f3_n, 20*np.log10(Sxx3_n), shading='gouraud')
ax1.set_title("phase 3 normal")
ax1.set_ylabel('Frequency [Hz]')
ax1.set_xlabel('Time [sec]')
fig.colorbar(im1).set_label('Intensity [dB]')
im1.set_clim(-400, -100)

ax2=fig.add_subplot(122)
im2=ax2.pcolormesh(t3_f, f3_f, 20*np.log10(Sxx3_f), shading='gouraud')
ax2.set_title("phase 3 faulty")
ax2.set_ylabel('Frequency [Hz]')
ax2.set_xlabel('Time [sec]')
fig.colorbar(im2).set_label('Intensity [dB]')
im2.set_clim(-400, -100)


In [ ]:
plot_n_signals(decompose_into_n_signals(phase1_faulty, 5))